# Assignment 5

For this assignment I took the csv files from the provided github repository. The I read them into pandas dataframe objects for analysis. 

In [1]:
import numpy as np
import pandas as pd
import datetime

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display_max_columns', 10)
pd.set_option('display_max_rows', 10)

df_airlines = pd.read_csv('airlines.csv')
df_airports = pd.read_csv('airports.csv')
df_planes = pd.read_csv('planes.csv')
df_weather = pd.read_csv('weather.csv')

df_airlines

ModuleNotFoundError: No module named 'numpy'